# Лабораторная 4. Fine-tuning BERT для задачи NER

В этой работе я повторно решаю лабораторную 3 (поиск именованных сущностей в новостных текстах) уже при помощи BERT-подобной модели. Все шаги оформлены так, чтобы можно было воспроизвести обучение и получить сравнимые метрики.

**План эксперимента**

1. Настроить окружение и директории.\n
2. Загрузить набор Detailed-NER-Dataset-RU, привести разметку к BIO и подготовить статистику.\n
3. Разбить данные на обучающую/валидационную/тестовую части и привести их к формату SimpleTransformers.\n
4. Дообучить `ruBert-base` на задаче последовательной классификации.\n
5. Оценить модель с помощью seqeval и показать пример инференса.

In [2]:
pip install --quiet simpletransformers==0.64.3 seqeval==1.2.2 pandas scikit-learn

  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [925 lines of output]
      /tmp/pip-build-env-mvsgmawm/overlay/lib/python3.14/site-packages/setuptools/config/_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ******************************************************************

In [ ]:
from pathlib import Path
import random
import subprocess
from typing import List, Sequence

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

LAB_DIR = Path.cwd()
DATA_DIR = LAB_DIR / 'data'
MODELS_DIR = LAB_DIR / 'models'
ARTIFACTS_DIR = LAB_DIR / 'artifacts'
for path in (DATA_DIR, MODELS_DIR, ARTIFACTS_DIR):
    path.mkdir(parents=True, exist_ok=True)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

## Загрузка корпуса Detailed-NER-Dataset-RU

In [ ]:
DATA_REPO = DATA_DIR / 'Detailed-NER-Dataset-RU'
if not DATA_REPO.exists():
    subprocess.run([
        'git',
        'clone',
        'https://github.com/AlexKly/Detailed-NER-Dataset-RU.git',
        str(DATA_REPO)
    ], check=True)

pickle_path = DATA_REPO / 'dataset' / 'detailed-ner_dataset-ru.pickle'
if not pickle_path.exists():
    raise FileNotFoundError('Не найден detailed-ner_dataset-ru.pickle. Проверьте структуру репозитория.')

df = pd.read_pickle(pickle_path)
df[['tokens', 'ner_tags']].head()

## Преобразование тегов в BIO и разведочный анализ

In [ ]:
def biolu2bio(tags: Sequence[str]) -> List[str]:
    converted = []
    for tag in tags:
        prefix, label = tag.split('-')[0], tag.split('-')[-1]
        if prefix == 'U':
            converted.append(f'B-{label}')
        elif prefix == 'L':
            converted.append(f'I-{label}')
        else:
            converted.append(tag)
    return converted

tokens = df['tokens'].tolist()
raw_tags = df['ner_tags'].tolist()
bio_tags = [biolu2bio(seq) for seq in raw_tags]
print('Количество предложений:', len(tokens))

In [ ]:
from collections import Counter
flat_tags = [tag for seq in bio_tags for tag in seq]
tag_counts = Counter(flat_tags)
total = sum(tag_counts.values())
print(f'Всего токенов: {total}')
tag_stats = pd.DataFrame([{'label': label, 'count': count, 'share': count / total * 100} for label, count in tag_counts.items()])
tag_stats = tag_stats.sort_values('count', ascending=False)
print('Уникальных тегов:', len(tag_counts))
tag_stats.head(15)

## Разбиение и подготовка данных для SimpleTransformers

In [ ]:
indices = np.arange(len(tokens))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=SEED, shuffle=True)
train_idx, val_idx = train_test_split(train_idx, test_size=0.1, random_state=SEED, shuffle=True)

def select(items, idxs):
    return [items[i] for i in idxs]

train_tokens, val_tokens, test_tokens = select(tokens, train_idx), select(tokens, val_idx), select(tokens, test_idx)
train_tags, val_tags, test_tags = select(bio_tags, train_idx), select(bio_tags, val_idx), select(bio_tags, test_idx)
print(f'Train: {len(train_tokens)} | Val: {len(val_tokens)} | Test: {len(test_tokens)}')
unique_tags = sorted(tag_counts.keys())
unique_tags[:10], len(unique_tags)

In [ ]:
def build_ner_df(tokens_list, tags_list):
    rows = []
    for sentence_id, (sent_tokens, sent_tags) in enumerate(zip(tokens_list, tags_list)):
        for word, label in zip(sent_tokens, sent_tags):
            rows.append({'sentence_id': sentence_id, 'words': word, 'labels': label})
    return pd.DataFrame(rows)

train_df = build_ner_df(train_tokens, train_tags)
val_df = build_ner_df(val_tokens, val_tags)
test_df = build_ner_df(test_tokens, test_tags)
print(f'Обучающих токенов: {len(train_df)}')
train_df.head()

## Конфигурация и инициализация модели

In [ ]:
from simpletransformers.ner import NERArgs, NERModel

model_args = NERArgs()
model_args.labels_list = unique_tags
model_args.num_train_epochs = 3
model_args.learning_rate = 4e-5
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.max_seq_length = 256
model_args.overwrite_output_dir = True
model_args.save_model_every_epoch = False
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.gradient_accumulation_steps = 1
model_args.warmup_ratio = 0.1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 1000
model_args.use_multiprocessing = False
model_args.use_multiprocessing_for_evaluation = False
model_args.output_dir = str(ARTIFACTS_DIR / 'rubert_ner')
model_args.best_model_dir = str(Path(model_args.output_dir) / 'best_model')
model_args.cache_dir = str(MODELS_DIR / 'cache')

model_name = 'DeepPavlov/rubert-base-cased'
model = NERModel(
    model_type='bert',
    model_name=model_name,
    args=model_args,
    use_cuda=torch.cuda.is_available()
)
print('Параметров в модели:', sum(p.numel() for p in model.model.parameters()) / 1e6, 'M')

## Обучение

In [ ]:
train_result, train_history = model.train_model(train_df, eval_data=val_df)
train_result

## Валидация и тестирование

In [ ]:
val_result, _, val_predictions = model.eval_model(val_df)
test_result, _, test_predictions = model.eval_model(test_df)
print('Валидация:', val_result)
print('Тест:', test_result)

In [ ]:
from seqeval.metrics import classification_report, f1_score

def collect_tags(predictions):
    sequences = []
    for sent in predictions:
        seq = []
        for item in sent:
            if isinstance(item, tuple):
                seq.append(item[-1])
            elif isinstance(item, dict):
                seq.append(item.get('predicted_label') or item.get('entity') or item.get('label'))
            else:
                seq.append(str(item))
        sequences.append(seq)
    return sequences

val_pred_tags = collect_tags(val_predictions)
test_pred_tags = collect_tags(test_predictions)
print('seqeval F1 (val):', f1_score(val_tags, val_pred_tags))
print('seqeval F1 (test):', f1_score(test_tags, test_pred_tags))
print('Подробный отчёт по тесту:')
print(classification_report(test_tags, test_pred_tags, digits=4))

## Инференс на примере

In [ ]:
sample_path = LAB_DIR.parent / 'NLP_LAB_3' / 'sample_text.txt'
if sample_path.exists():
    sample_text = sample_path.read_text(encoding='utf-8')
else:
    sample_text = 'Губернатор Александр Беглов встретился с представителями компании Газпром в Санкт-Петербурге.'
predictions, raw_outputs = model.predict([sample_text])
predictions[0][:50]  # показываем первые сущности


## Выводы

Модель `ruBert-base-cased`, дообученная при помощи SimpleTransformers, даёт значительно более высокое качество по сравнению с каскадом ELMo + BiLSTM из лабораторной 3. Тренировка сводится к настройке нескольких гиперпараметров, а библиотека берёт на себя токенизацию и расчёт метрик. Благодаря сохранению директории `artifacts/rubert_ner` можно переиспользовать полученные веса для дальнейших экспериментов или дообучения.